In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt update
!sudo apt install p7zip-full -y

In [ ]:
!git clone https://github.com/Kuugo2002/svc-fusion-linux.git

In [ ]:
import requests
import os
from tqdm import tqdm

def download_file(
    url: str,
    save_path: str = None,
    save_dir: str = ".",
    filename: str = None,
    chunk_size: int = 8192,
    show_progress: bool = True
) -> bool:
    """
    带进度条的文件下载函数，可灵活指定保存位置
    
    :param url: 文件下载URL（必需）
    :param save_path: 完整保存路径（包含文件名），优先级最高
    :param save_dir: 保存目录（当save_path未指定时使用）
    :param filename: 保存文件名（当save_path未指定时使用）
    :param chunk_size: 下载分块大小（字节）
    :param show_progress: 是否显示进度条
    :return: 下载成功返回True，否则False
    """
    try:
        # 确定最终保存路径
        if save_path is None:
            if filename is None:
                filename = url.split('/')[-1].split('?')[0]  # 去除URL参数
            save_path = os.path.join(save_dir, filename)
        
        # 标准化路径（处理~和相对路径）
        save_path = os.path.abspath(os.path.expanduser(save_path))
        save_dir = os.path.dirname(save_path)
        
        # 发起HEAD请求获取文件信息
        response = requests.head(url, allow_redirects=True)
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        
        # 检查文件是否已存在（支持断点续传）
        if os.path.exists(save_path):
            downloaded_size = os.path.getsize(save_path)
            if downloaded_size == total_size:
                print(f"文件已存在且完整: {save_path}")
                return True
            headers = {'Range': f'bytes={downloaded_size}-'}
            resume = True
        else:
            downloaded_size = 0
            headers = None
            resume = False
        
        # 发起下载请求
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        
        # 设置进度条
        progress_bar = None
        if show_progress:
            progress_bar = tqdm(
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
                desc=os.path.basename(save_path),
                initial=downloaded_size,
                ncols=80,
                ascii=True,
                colour='green'
            )
        
        # 下载文件
        mode = 'ab' if resume else 'wb'
        with open(save_path, mode) as f:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:  # 过滤掉保持连接的空块
                    f.write(chunk)
                    if progress_bar:
                        progress_bar.update(len(chunk))
        
        if progress_bar:
            progress_bar.close()
        
        # 验证下载完整性
        if total_size != 0 and os.path.getsize(save_path) != total_size:
            print(f"警告: 下载文件大小与预期不符 ({os.path.getsize(save_path)}/{total_size} bytes)")
        
        print(f"文件已保存到: {save_path}")
        return True
    
    except requests.exceptions.RequestException as e:
        print(f"\n下载失败: {str(e)}")
        if progress_bar:
            progress_bar.close()
        return False
    except Exception as e:
        print(f"\n发生意外错误: {str(e)}")
        if progress_bar:
            progress_bar.close()
        return False

download_file("https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/env.7z")

In [ ]:
import subprocess
def extract_in_background(archive_path, output_dir, threads="on"):
    # 启动 7z 解压（后台运行）
    process = subprocess.Popen(
        [
            "7z",
            "x",
            archive_path,
            f"-o{output_dir}",
            f"-mmt={threads}",
            "-y",  # 自动确认所有提示
        ],
        stderr=subprocess.PIPE,
        text=True,
    )
    return process

print("开始解压python环境")
process = extract_in_background(
    "/content/env.7z",
    "/content/svc-fusion-linux",
    threads="on"  # 自动检测线程数
)


In [ ]:
dir="/content/svc-fusion-linux/other_weights"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/other_weights/KimMelBandRoformer.ckpt",save_dir=dir )
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/other_weights/deverb_bs_roformer_8_256dim_8depth.ckpt",save_dir=dir )
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/other_weights/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt",save_dir=dir )

dir="/content/svc-fusion-linux/pretrain/contentvec"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrain/contentvec/checkpoint_best_legacy_500.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrain/fcpe"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrain/fcpe/fcpe.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrain/hubert"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrain/hubert/hubert-soft-0d54a1f4.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrain/nsf_hifigan"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrain/nsf_hifigan/config.json",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrain/nsf_hifigan/model",save_dir=dir)

dir="/content/svc-fusion-linux/pretrain/rmvpe"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrain/rmvpe/model.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6/contentvec768l12.0"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6/contentvec768l12.0/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6/contentvec768l12.0/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6/contentvec768l12.0/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/185c2de936da1260eb212b7482b509d6"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/185c2de936da1260eb212b7482b509d6/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/185c2de936da1260eb212b7482b509d6/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/185c2de936da1260eb212b7482b509d6/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/5a733e323c0e6a0673b457acdb9e3bec"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/5a733e323c0e6a0673b457acdb9e3bec/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/5a733e323c0e6a0673b457acdb9e3bec/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/5a733e323c0e6a0673b457acdb9e3bec/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/5d4e8f22abefd670aa2d0c0924c7c122"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/5d4e8f22abefd670aa2d0c0924c7c122/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/5d4e8f22abefd670aa2d0c0924c7c122/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/5d4e8f22abefd670aa2d0c0924c7c122/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/74debe07c939fb9fecb2835935a8102c"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/74debe07c939fb9fecb2835935a8102c/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/74debe07c939fb9fecb2835935a8102c/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/74debe07c939fb9fecb2835935a8102c/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/b43e6e57379a9e387e3ab18faa6a11c7"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/b43e6e57379a9e387e3ab18faa6a11c7/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/b43e6e57379a9e387e3ab18faa6a11c7/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/b43e6e57379a9e387e3ab18faa6a11c7/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/c76f3fca6b0a7e29391ed32bf390751a"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/c76f3fca6b0a7e29391ed32bf390751a/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/c76f3fca6b0a7e29391ed32bf390751a/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/c76f3fca6b0a7e29391ed32bf390751a/model_0.pt",save_dir=dir)

dir="content/svc-fusion-linux/pretrained/ddsp6.1/cbeeab26100ed0170a86630000506d63"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/cbeeab26100ed0170a86630000506d63/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/cbeeab26100ed0170a86630000506d63/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/cbeeab26100ed0170a86630000506d63/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/d4d56869daf7f4c5ecf609a9087f0da7"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/d4d56869daf7f4c5ecf609a9087f0da7/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/d4d56869daf7f4c5ecf609a9087f0da7/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/d4d56869daf7f4c5ecf609a9087f0da7/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/ddsp6.1/eed85868e711a53f48bbe17d4426a26e"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/eed85868e711a53f48bbe17d4426a26e/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/eed85868e711a53f48bbe17d4426a26e/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/ddsp6.1/eed85868e711a53f48bbe17d4426a26e/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/reflow/contentvec768l12"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/reflow/contentvec768l12/config.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/reflow/contentvec768l12/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/reflow/contentvec768l12/model_0.pt",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/sovits/vec768l12"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/sovits/vec768l12/D_0.pth",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/sovits/vec768l12/G_0.pth",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/sovits/vec768l12/meta.yaml",save_dir=dir)

dir="/content/svc-fusion-linux/pretrained/sovits_diff/vec768l12"
os.makedirs(dir, mode=0o755, exist_ok=True)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/sovits_diff/vec768l12/meta.yaml",save_dir=dir)
download_file(url="https://huggingface.co/Kuugo/svc-fusion-linux-model/resolve/main/pretrained/sovits_diff/vec768l12/model_0.pt",save_dir=dir)

In [ ]:
print("等待解压完成...")
return_code = process.wait()  # 阻塞，直到解压完成

if return_code == 0:
    print("解压成功！")
else:
    print(f"解压失败，错误码: {return_code}")
    print(process.stderr.read())

In [ ]:
!ln -s /content/drive/MyDrive/data /content/svc-fusion-linux
!ln -s /content/drive/MyDrive/dataset_raw /content/svc-fusion-linux
!ln -s /content/drive/MyDrive/results /content/svc-fusion-linux
!ln -s /content/drive/MyDrive/exp /content/svc-fusion-linux

In [ ]:
!chmod +x /content/svc-fusion-linux/env/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2
!chmod +x /content/svc-fusion-linux/env/bin/python
!/content/svc-fusion-linux/env/bin/python /content/svc-fusion-linux/launcher.py